# Machine Learning (ML) Workflows


Stepping into the world of ML is an exciting journey, but it often comes with complexities that can hinder innovation and experimentation. A workflow solutions to solve many of these issues, offering tools and simplifying processes to streamline the ML lifecycle and foster collaboration.

## MLflow Overview
Whether you’re an individual researcher, a member of a large team, or somewhere in between, MLflow provides a unified platform to navigate the intricate maze of model development, deployment, and management. MLflow aims to enable innovation in ML solution development by streamlining otherwise cumbersome logging, organization, and lineage concerns that are unique to model development. This focus allows you to ensure that your ML projects are robust, transparent, and ready for real-world challenges.

MLflow, at its core, provides a suite of tools aimed at simplifying the ML workflow. It is tailored to assist ML practitioners throughout the various stages of ML development and deployment. Despite its expansive offerings, MLflow’s functionalities are rooted in several foundational components:

- **Tracking**: MLflow Tracking provides both an API and UI dedicated to the logging of parameters, code versions, metrics, and artifacts during the ML process. This centralized repository captures details such as parameters, metrics, artifacts, data, and environment configurations, giving teams insight into their models’ evolution over time. Whether working in standalone scripts, notebooks, or other environments, Tracking facilitates the logging of results either to local files or a server, making it easier to compare multiple runs across different users.
- **Model Registry**: A systematic approach to model management, the Model Registry assists in handling different versions of models, discerning their current state, and ensuring smooth productionization. It offers a centralized model store, APIs, and UI to collaboratively manage an MLflow Model’s full lifecycle, including model lineage, versioning, aliasing, tagging, and annotations.
- **MLflow Deployments for LLMs**: This server, equipped with a set of standardized APIs, streamlines access to both SaaS and OSS LLM models. It serves as a unified interface, bolstering security through authenticated access, and offers a common set of APIs for prominent LLMs.
- **Evaluate**: Designed for in-depth model analysis, this set of tools facilitates objective model comparison, be it traditional ML algorithms or cutting-edge LLMs.
- **Prompt Engineering UI**: A dedicated environment for prompt engineering, this UI-centric component provides a space for prompt experimentation, refinement, evaluation, testing, and deployment.
- **Recipes**: Serving as a guide for structuring ML projects, Recipes, while offering recommendations, are focused on ensuring functional end results optimized for real-world deployment scenarios.
- **Projects**: MLflow Projects standardize the packaging of ML code, workflows, and artifacts, akin to an executable. Each project, be it a directory with code or a Git repository, employs a descriptor or convention to define its dependencies and execution method.

![mlflow-overview](../../../images/wsl-jupyter-mlflow-overview.png)

## Running with Tracking Server
First of all, make sure that your local MLflow Tracking Service is running, If not, run the `mlflow server` CLI command in a new terminal with the virtual environment activated. By default, the tracking server will be running at `http://localhost:5000`. We'll use the MLflow fluent API to perform all interactions with the tracking server. Make sure your tracking URI is set correctly and replace the `tracking_uri` with your own URI if you are using other mlflow tracker server such as the free Databricks Community Edition.

In [1]:
import mlflow

tracking_uri = "http://localhost:5000"
mlflow.set_tracking_uri(uri=tracking_uri)

### Define an MLflow Experiment
In order to group any distinct runs of a particular project or idea together, we can define an Experiment that will group each iteration (runs) together. Defining a unique name that is relevant to what we're working on helps with organization and reduces the amount of work (searching) to find our runs later on. 

In [2]:
# set a new experiment to avoid
# cluttering the default experiment
experiment = mlflow.set_experiment("mlflow-tracking-quickstart")

In [3]:
# logistic regression experiment
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from mlflow.models import infer_signature

# load the Iris dataset
ds = datasets.load_iris(as_frame=True)
display(ds.frame)

X, y = datasets.load_iris(return_X_y=True)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [4]:
# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# define the model hyperparameters
params = {"solver": "lbfgs", "max_iter": 1000, "multi_class": "auto", "random_state": 8888}

# train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# predict on the test set
y_pred = lr.predict(X_test)

# calculate accuracy as a target loss metric
accuracy = accuracy_score(y_test, y_pred)

/home/pen/ml-lab/data-lab-on-wsl/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


### Log the model, hyperparameters, and loss metrics to MLflow Tracking Server
In order to record our model and the hyperparameters that were used when fitting the model, as well as the metrics associated with validating the fit model upon holdout data, we initiate a run context, as shown below. Within the scope of that context, any fluent API that we call (such as `mlflow.log_params()` or `mlflow.sklearn.log_model()`) will be associated and logged together to the same run.

In [5]:
# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2025/02/06 21:58:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 5


🏃 View run skillful-ant-555 at: http://localhost:5000/#/experiments/963922167585500197/runs/d15a72fc63814d6a8993515390809945
🧪 View experiment at: http://localhost:5000/#/experiments/963922167585500197


Created version '5' of model 'tracking-quickstart'.


![mlflow-ui-experiment-list](../../../images/wsl-jupyter-mlflow-ui-exp-list.png)
![mlflow-ui-experiment-details-overview](../../../images/wsl-jupyter-mlflow-ui-exp-overview.png)
![mlflow-ui-experiment-details-artifact](../../../images/wsl-jupyter-mlflow-ui-exp-artifact.png)

### Load the artifact as a Python Function
Although you can load our model back as a native scikit-learn format with `mlflow.sklearn.load_model()`, below you are loading the model as a generic Python Function, which is how this model would be loaded for online model serving. You can still use the `pyfunc` representation for batch use cases, though, as is shown below.

In [6]:
# load saved model from artifact as a python function for model serving
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

/home/pen/ml-lab/data-lab-on-wsl/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Prediction using the loaded model
Use the model trained and registerd in the previous step to predict the iris class type on a Pandas DataFrame.

In [7]:
# run prediction using the loaded model
predictions = loaded_model.predict(X_test)
iris_feature_names = datasets.load_iris().feature_names

# convert X_test validation feature data to a Pandas DataFrame
result = pd.DataFrame(X_test, columns=iris_feature_names)

# add the actual classes to the DataFrame
result["actual_class"] = y_test

# add the model predictions to the DataFrame
result["predicted_class"] = predictions

result[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1


## Terminate Tracking Server
To terminate the local MLflow Tracking Server, press `ctrl+c` on the terminal where the tracking server running. If you are using a remote tracker server, you can skip this step.

# Additional Resources
- For this example, we're using a locally running tracking server, but other options are available (The easiest is to use the free managed service within [Databricks Community Edition](https://community.cloud.databricks.com/)).

# References

- [MLflow Overview](https://mlflow.org/docs/latest/introduction/index.html)